# **iMaterialist Challenge (Furniture) at FGVC5**
## **TFNW Kaggle Team**

### **Get Started**

To get started, you will:



1.   Join the Completion
2.   Create a Kaggle API key
3.   Install the Kaggle Python module
4.   Download the dataset for the competition

**Join The Completion**

Goto Kaggle.com and login.

Once logged in, goto the competition page and select the JOIN COMPETITION button

https://www.kaggle.com/c/imaterialist-challenge-furniture-2018

**Create Kaggle API Key**

To download this dataset you will use the Kaggle API for downloading datasets. To use the API, you must first create a Kaggle API Key. To create your API Key, do:



1.   Click on your Profile
2.   Select My Accont
3.   Under API, Select Create New API Token
4.   Download the API Key (kaggle.json)
5.   Copy the API key to:  ~/.kaggle/kaggle.json
6.   On Windows, that would be: \Users\<username>\.kaggle\kaggle.json

**Install the Kaggle Python Module**

C:> pip install kaggle




In [1]:
# Get the latest version of pip
!python -m pip install --upgrade pip

# Install the Kaggle module
!pip install kaggle

Requirement already up-to-date: pip in c:\users\user\appdata\local\programs\python\python35\lib\site-packages




### Download the Dataset Dictionary

The dataset consists of a dataset dictionary and the data. The dataset dictionary is a JSON file that contains the URL location and label for each image in the dataset. We need to download this first using the Kaggle python module:

C:> kaggle competitions download -c imaterialist-challenge-furniture-2018

This will place the data under:

Linux/Mac: ~/.Kaggle/competitions/imaterialist-challenge-furniture-2018<br/>
Windows: \Users\<username>\.Kaggle\competitions\imaterialist-challenge-furniture-2018

In [2]:
!kaggle competitions download -c imaterialist-challenge-furniture-2018

train.json: Skipping, found more recently modified local copy (use --force to force download)
validation.json: Skipping, found more recently modified local copy (use --force to force download)
test.json: Skipping, found more recently modified local copy (use --force to force download)
sample_submission_randomlabel.csv: Skipping, found more recently modified local copy (use --force to force download)


Next, we will install and import some libraries.

In [3]:
# numpy for the high performance in-memory matrix/array storage and operations.
!pip install numpy   
# h5py for the HD5 filesystem high performance file storage of big data.
!pip install h5py   
# Python image manipulation library (replaces PIL)
!pip install pillow  
# requests for HTTP operations
!pip install requests

In [4]:
# Import numpy for the high performance in-memory matrix/array storage and operations.
import numpy as np

# Import h5py for the HD5 filesystem high performance file storage of big data.
import h5py

# Import PIL.Image for Python image manipulation library. 
from PIL import Image

# Import json and requests for HTTP operations
import json, requests

# Import the Byte and String IO library for extracing data returned (response) frome HTTP requests.
from io import BytesIO, StringIO

# Import time to record timing
import time


c:\users\user\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# **Loading the Dataset (ETL)**

## ** Overview **

The loading and storing of the image data and corresponding data is done in several steps. To facilate minimizing the loading time and memory requirements, the process is broken into batches, and loaded and stored in concurrent (parallel) groups. 

Both the size of batches and the number of concurrent load/stores of batches is configurable. The overall process is as follows:

1. Load the Dataset Dictionary (described below) for the training (or test or validation) data.
2. Determine the number of images to load/store from the data dictionary.
3. Split the loading/storing of images into batches, based on batch size.
4. Sequentially load groups of batches, but where each batch within the batch group is concurrently processed (loaded and stored).

## ** Dispatch **

The function load_dispatcher() handles the dispatching of loading/storing of batches. It starts by
taking the location of the dataset dictionary and loads it into memory. The dataset dictionary contains the location of each image and corresponding label.

The dataset dictionary is in json format, as follows, where [image] is a list of image locations and [annotations] is a list of corresponding labels.
 
 {<br/>
"images" : [image],<br/>
"annotations" : [annotation],<br/>
}

The dispatcher makes a HTTP request (requests.get(url)) for the dataset dictionary and extracts the dataset dictionary from the contents of the response (requests.get(url).content). The dispatcher extracts it as raw byte data (ByteIO) and loads the dataset dictionary into a json format (json.load).
 
The dispatcher calculates the number of batches based on the number of images and the batch size. The batches are then sequenced into groups (for i in range(0, batches, concurrent)), where the size of the group is the number of concurrent threads. For each group, the dispatcher creates a thread for each batch to load/store asynchronously (i.e., job). The dispatcher waits for all the concurrent jobs to complete. Based on the time to process the batch group, the dispatcher estimates the time to load/store all the remaining batches.

Note, you can set the size of the batches, the size to rescale the images to, and whether to convert to grayscale.

In [5]:
# Library for thread execution
import threading

def load_dispatcher(url, batch_type, batch_size=200, size=(300,300), grayscale=False, concurrent=5):
    """ Load the Data in Batches 
    url - location of data dictionary
    batch_type - training, validation or test
    batch_size - size of the batch
    size - size to rescale image to
    grayscale - flag to convert image to grayscale
    concurrent - the number of concurrent (parallel)) batch loads
    """
    
    # First retreive the dataset dictionary, which is in a JSON format. 
    # Dictionary is stored remote: We will make a HTTP request
    if url.startswith("http"):
        datadict = json.load( requests.get(url).content )
    # Dictionary is stored locally
    else:
        datadict = json.load( open( url ) )
   
    # The number of batches
    batches = int(len(datadict['images']) / batch_size)
    
    # Sequentially Load each batch group (i.e., concurrent)
    for i in range(0, batches, concurrent):
        # Start time for the batch group
        start_time = time.time()
        
        # Listof threads, corresponding to to the processing of each batch in the batch group
        threads = []
        # Create and Start a processing thread for each batch in the batch group
        for j in range(concurrent):
            t = threading.Thread(target=load_and_store_batch, args=(datadict, batch_type, i + j, batch_size, size, grayscale,))
            # Keep track (remember) of the thread
            threads.append(t)
            # Start the thread
            t.start()
        # Join the threads into a single wait for all threads to complete
        for t in threads:
            t.join()
                  
        # Calculate elapsed time in seconds to load this batch group
        elapse = int(time.time() - start_time)
            
        # Estimate remaining time in minutes for loading remaining barches.
        remaining = int( ( ( batches - i ) / concurrent ) * elapse ) / 60
        
        print("Remaining time %d mins" % remaining)

## ** Load and Store the Data Batches **

We have a mass amount of images: 194,828 training, 6,400 validation, 12,800 test. If we tried to load it all, we would need 54GB of memory! 

Instead, we will load the data into smaller training (validation and test) batches and store them separately on disk. We use the loaded data dictionary and sequentially move through it (batch_size at a time) to build the batches and save them into a HD5 high performance file system.

In [6]:
def load_and_store_batch(datadict, batch_type, pos, batch_size, size, grayscale):
    """ Process loading (extration), handling (transformation) and storing (loading) as a batch 
    batch_type - training, validation or test
    pos - the batch slice position in the data (i.e., the first, the second, etc)
    batch_size - size of the batch
    size - size to rescale image to
    grayscale - flag to convert image to grayscale
    """
    start_time = time.time()
    
    start = pos * batch_size
    images, labels = load_batch(datadict, start, batch_size, size, grayscale )
        
    # Calculate elapsed time in seconds to load this batch
    elapse = int(time.time() - start_time)
        
    print("Batch Loaded %d: %d secs" % (pos, elapse))
        
    # Write the batch to disk as HD5 file
    with h5py.File('contents\\' + batch_type + '\\images' + str(pos) + '.h5', 'w') as hf:
        hf.create_dataset("images",  data=images)
    with h5py.File('contents\\' + batch_type + '\\labels' + str(pos) +  '.h5', 'w') as hf:
        hf.create_dataset("labels",  data=labels)

## ** Extraction / Dataset Directory **

The load_batch() function uses the location of each image specified in the slice of the dataset dictionary to load the images into memory, do transformations and store the images on disk, as a batch. The slice is defined by a start position in the dataset dictionary and length, denoted by batch_size.

## **Transform / Load**
 
The images are all RGB images, but are of different pixel sizes. For the neural network, they all need to be the same size. We will rescale each of our images to be 300 by 300 pixels (default), but you can choose another scale with the size parameter. THe image data will then be packed into a high performance numpy 3D matrix. The row/column are the height and width (300,300) and the third dimension are the channels (3).

The images will be stored in the list images[] and the corresponding labels in the list labels[].

The load_batch() function loads a batch of images from the training, validation or test data and does the transform function. The images and corresponding labels are then returned as a list.

In [7]:
def load_batch(datadict, start, batch_size=200, size=(300,300), grayscale=False ):
    """ Load the training dataset 
    datadict - data image/label dictionary
    start - index to start reading batch of images
    batch_size - number of images to read (None = all images)
    grayscale - flag if image should be converted to grayscale
    """
    
    images = [] # List containing the images
    labels = [] # List containing the corresponding labels for the images
    
    # Number of images to load
    if batch_size == None:
        batch_size = len(datadict['images'])
      
    # Final shape of image Height, Width
    if grayscale == True:
        shape = size
    # Final shape of image Height, Width, Channels(3)
    else:
        shape = size + (3,)
            
    # Load the batch of images/labels from the Data Dictionary
    end = start + batch_size
    for i in range(start, end): 
        image_url = datadict['images'][i]['url'][0]
        label_id  = datadict['annotations'][i]['label_id']
        
        not_loaded = 0

        # Download, resize and convert images to arrays
        try:
            # Make HTTP request fot the image data
            response = requests.get(image_url, timeout=5)
            
            # Use the PIL.Image libary to load the image data as au uncompressed RGB or Grayscale bitmap
            if grayscale == True:
                image = Image.open(BytesIO(response.content)).convert('LA')
            else:
                image = Image.open(BytesIO(response.content))
            
            # Resize the image to be all the same size
            image = image.resize(size, resample=Image.LANCZOS)
            
            # Load the image into a 3D numpy array
            image = np.asarray(image)
            
            # Discard image if it does not fit the final shape
            assert image.shape == shape
        except Exception as ex:
            not_loaded += 1
            continue
        
        # if bad image, skip
        if np.any(image == None):
            continue
        # add image to images list
        images.append( image )
        # add corresponding label to labels list
        labels.append( label_id )
        
        if (i+1) % 50 == 0:
            print('%d Images added, %d not loaded' % ((i + 1), not_loaded))

    return images, labels
        

## Execute the Load

The URLs below are for more laptop. You will need to modify it to the location on your laptop.

Running as 5 concurrent processes in batches of 200, takes 1/2 day on my laptop with my local Internet service.

In [ ]:
# Create Directories for the HD5 encoded batches
!mkdir contents
!mkdir contents\\train
!mkdir contents\\validation
!mkdir contents\\test

# Data dictionaries
train_url      = 'C:\\Users\\User\\.kaggle\\competitions\\imaterialist-challenge-furniture-2018\\train.json'
test_url       = 'C:\\Users\\User\\.kaggle\\competitions\\imaterialist-challenge-furniture-2018\\test.json'
validation_url = 'C:\\Users\\User\\.kaggle\\competitions\\imaterialist-challenge-furniture-2018\\validation.json'

# Load the Training Batches
load_dispatcher(train_url, "train")

# Load the Validation Batches
load_dispatcher(validation_url, "validation")

# Load the Test Batches
load_dispatcher(test_url, "test")

250 Images added, 0 not loaded
50 Images added, 0 not loaded
650 Images added, 0 not loaded
450 Images added, 0 not loaded
850 Images added, 0 not loaded
100 Images added, 0 not loaded
300 Images added, 0 not loaded
700 Images added, 0 not loaded
500 Images added, 0 not loaded
900 Images added, 0 not loaded
150 Images added, 0 not loaded
750 Images added, 0 not loaded
950 Images added, 0 not loaded
350 Images added, 0 not loaded
550 Images added, 0 not loaded
200 Images added, 0 not loaded
Batch Loaded 0: 198 secs
1000 Images added, 0 not loaded
Batch Loaded 4: 243 secs
400 Images added, 0 not loaded
Batch Loaded 1: 244 secs
600 Images added, 0 not loaded
Batch Loaded 2: 249 secs
800 Images added, 0 not loaded
Batch Loaded 3: 267 secs
Remaining time 866 mins
1450 Images added, 0 not loaded
1050 Images added, 0 not loaded
1250 Images added, 0 not loaded
1650 Images added, 0 not loaded
1850 Images added, 0 not loaded
1500 Images added, 0 not loaded
1300 Images added, 0 not loaded
1900 Im

c:\users\user\appdata\local\programs\python\python35\lib\site-packages\PIL\TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 9830400 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\user\appdata\local\programs\python\python35\lib\site-packages\PIL\TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


10150 Images added, 0 not loaded
10950 Images added, 0 not loaded
10750 Images added, 0 not loaded
10550 Images added, 0 not loaded
10200 Images added, 0 not loaded
Batch Loaded 50: 251 secs
11000 Images added, 0 not loaded
Batch Loaded 54: 260 secs
10800 Images added, 0 not loaded
Batch Loaded 53: 266 secs
10600 Images added, 0 not loaded
Batch Loaded 52: 276 secs
10400 Images added, 0 not loaded
Batch Loaded 51: 277 secs
Remaining time 853 mins
11450 Images added, 0 not loaded
11650 Images added, 0 not loaded
11850 Images added, 0 not loaded
11050 Images added, 0 not loaded
11500 Images added, 0 not loaded
11900 Images added, 0 not loaded
11700 Images added, 0 not loaded
11300 Images added, 0 not loaded
11100 Images added, 0 not loaded
11750 Images added, 0 not loaded
11550 Images added, 0 not loaded
11350 Images added, 0 not loaded
11150 Images added, 0 not loaded
Batch Loaded 59: 3304 secs
11800 Images added, 0 not loaded
Batch Loaded 58: 3324 secs
11600 Images added, 0 not loaded


Remaining time 1182 mins
21450 Images added, 0 not loaded
21250 Images added, 0 not loaded
21650 Images added, 0 not loaded
21850 Images added, 0 not loaded
21500 Images added, 0 not loaded
21700 Images added, 0 not loaded
21900 Images added, 0 not loaded
21050 Images added, 0 not loaded
21550 Images added, 0 not loaded
21300 Images added, 0 not loaded
21750 Images added, 0 not loaded
21950 Images added, 0 not loaded
21100 Images added, 0 not loaded
21350 Images added, 0 not loaded
22000 Images added, 0 not loaded
Batch Loaded 109: 234 secs
21600 Images added, 0 not loaded
Batch Loaded 107: 236 secs
21800 Images added, 0 not loaded
Batch Loaded 108: 281 secs
21400 Images added, 0 not loaded
Batch Loaded 106: 298 secs
21150 Images added, 0 not loaded
21200 Images added, 0 not loaded
Batch Loaded 105: 398 secs
Remaining time 1152 mins
22250 Images added, 0 not loaded
22650 Images added, 0 not loaded
22450 Images added, 0 not loaded
22050 Images added, 0 not loaded
22850 Images added, 0 n